In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


running k-means on cuda:0..


[running kmeans]: 14it [00:00, 68.48it/s, center_shift=0.000060, iteration=14, tol=0.000100]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [2]:
import importlib
importlib.reload(cluster_model)

# TODO: auto sim cutoff needed
decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=2)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Trying 50 clusters for layer blocks.0.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 84it [00:30,  2.75it/s, center_shift=0.000000, iteration=84, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Trying 50 clusters for layer blocks.1.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 77it [00:28,  2.71it/s, center_shift=0.000000, iteration=77, tol=0.000100]  


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Trying 50 clusters for layer blocks.2.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


...

CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.4.hook_resid_post
Trying 50 clusters for layer blocks.4.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 97it [00:36,  2.69it/s, center_shift=0.000000, iteration=97, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Finding optimal number of clusters and such clusters for layer blocks.5.hook_resid_post
Trying 50 clusters for layer blocks.5.hook_resid_post
USING DEVICE cuda:0
running k-means on cuda:0..


[running kmeans]: 40it [00:14,  2.69it/s, center_shift=0.000000, iteration=40, tol=0.000100]   


CLUSTER SHAPE torch.Size([50, 512])
Getting cluster scores for lattice
Getting scores for layers 0 to 1
Getting scores for layers 1 to 2
Getting scores for layers 2 to 3
Getting scores for layers 3 to 4
Getting scores for layers 4 to 5
CLUSTER SCORES [array([[3.49755864, 4.09916179, 4.38130361, ..., 3.61500923, 4.15072528,
        4.40484566],
       [3.92474343, 4.65688239, 5.11339251, ..., 4.11866036, 4.78939758,
        5.10040761],
       [3.8695757 , 4.57967225, 4.99839566, ..., 4.04940191, 4.69246571,
        4.99108061],
       ...,
       [3.65260714, 4.33045255, 4.77294097, ..., 3.83886386, 4.4593576 ,
        4.74419257],
       [3.52389   , 4.17564074, 4.5127983 , ..., 3.68824032, 4.24417722,
        4.52501681],
       [3.47816673, 4.11734485, 4.50113336, ..., 3.64116984, 4.22118926,
        4.49333164]]), array([[0.70333037, 3.92304845, 0.87358022, ..., 3.83998436, 3.70563864,
        3.76602658],
       [0.81791344, 4.6243185 , 0.91250215, ..., 4.54167395, 4.38050541,
   

In [3]:
decomp.lattice_scores[0].shape

(50, 50)

In [4]:
from typing import List
import utils
importlib.reload(utils)
modified_lattice = decomp.lattice_scores.copy()

# TODO: have function which does something like fixing from the a specific neuron?
# Ie have the sub-lattice with only **related** neurons

def cutoff_lattice(lattice: List[List[List[float]]], related_cutoff = 1):
    print(lattice[0].sum())
    r = [(layer > related_cutoff) * layer for layer in lattice]
    print(r[0].sum())
    return r

def create_sublattice(lattice, layer: int, idx_in_layer: int, related_cutoff = 10):
	with_cutoff = cutoff_lattice(lattice, related_cutoff)

LAST_FEAT = 20
modified_lattice = cutoff_lattice(decomp.lattice_scores, related_cutoff=15)

# utils.top_k_dag_paths_dynamic(modified_lattice, k=10_000)

10436.00001455494
200.5030490747813


In [5]:
# TODO:
""""
We need to combine the 
"""

import numpy as np
from utils import get_random_cutoff

score_path = [8, 57, 89]
score_path = [10, 63, 89]
score_path = [3, 3, 1]
# score_path = [39, 14, 2, 3, 4]
score_path = [1, 12, 2, 3, 4, 5]
"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space
"""

to_score = [get_random_cutoff(d, 100) for d in ds][:300]
scores = decomp.score(
    to_score,
    score_path=score_path,
    weighting_per_layer=np.array([1, 1, 1, 1, 1, 1])
)


GOT EMBEDS (6, 5412, 512)
(5412, 6, 512) [1, 12, 2, 3, 4, 5]
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTANCES
USING DISTA

In [13]:
min([min(s) for s in scores]), max([max(s) for s in scores])

(13018.508758544922, 38539.830741882324)

In [16]:
from IPython.core.display import display, HTML
import math
import numpy as np

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord
act_simp = [[[[0 if math.isnan(tok * 100) else int(math.exp(tok * 0.001))]] for tok in s] for s in scores_reord]
# TODO sep fun
from circuitsvis.utils.render import RenderedHTML, render
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))

/tmp/ipykernel_50053/4091926540.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Isolate Specific Neurons

In [18]:
import os
import cluster_model
importlib.reload(cluster_model)

LAYER = 3
NODE = 10
# TODO: this ain't right!! We need specific paths!!
ret = cluster_model.restrict_to_related_path(decomp.lattice_scores, LAYER, NODE, n_paths=10_000)
# Write to file
with open("/tmp/related_paths.txt", "w") as f:
    f.write("\n".join([str(r) for r in ret]))
len(ret)

10000

## Try different scoring method

In [ ]:
# importlib.reload(cluster_model)

# score_path = [0, 16, 10, 10, 31, 8]
# # TODO: embeds to here...
# importlib.reload(cluster_model)
# _, top_ds, scores = cluster_model.score_for_neuron(decomp, to_score[:200], LAYER, score_path, top_n=20, weighting_per_layer=[0, 1, 1, 1, 1, 1])

## Try more path like method

In [ ]:
importlib.reload(cluster_model)
score_path = [35, 16, 6, 10, 11, 32]

"""
Paths which seem to have meaning

score_path = [1, 1, 2, 3, 4]
Very strong for Q: . Specifically on the space

score_path_path = [3, 15, 19, 11, 12]
Very strong for Q: . when context is about *programming*
"""

weighting_per_layer = np.ones(len(decomp.lattice_scores) + 1)
weighting_per_layer[LAYER] = 10
weighting_per_layer[0] = 0
weighting_per_layer[1] = 0

to_score = [utils.get_random_cutoff(d, 200) for d in ds][:600]
scores = decomp.score(
    to_score,
    score_path=score_path,
    # TODO: how do we weight???
    weighting_per_layer=weighting_per_layer
)

scores_per_token_set = np.array([max(s) for s in scores])
top_args = np.argsort(scores_per_token_set)[::-1]

tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)[
    'input_ids']] for d in to_score]
tokens_reord = [tokens[i] for i in top_args]
scores_reord = [scores[i] for i in top_args]
# TODO: SORT!
act_simp = scores_reord

GOT EMBEDS (6, 10157, 512)
(10157, 6, 512) [9, 16, 10, 10, 42, 31]


AttributeError: 'KMeansMixture' object has no attribute 'distances_squared'

In [ ]:
act_simp = [[[[0 if math.isnan(tok * 10) else int(math.exp(tok * 40))]]
             for tok in s] for s in scores_reord]
# TODO sep fun
html = render(
    "TextNeuronActivations",
    tokens=tokens_reord,
    activations=act_simp,
    firstDimensionName="Layer",
    secondDimensionName="Neuron",
    firstDimensionLabels=None,
    secondDimensionLabels=None
)
display(HTML(str(html)))